<a href="https://colab.research.google.com/github/aryan1429/Big-Data/blob/main/MatrixMultiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless > /dev/null
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz
!mv hadoop-3.3.6 /usr/local/hadoop


In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["PATH"] += f":{os.environ['HADOOP_HOME']}/bin:{os.environ['HADOOP_HOME']}/sbin"

!hadoop version


In [ ]:
!echo "This is a sample file for Hadoop testing." > sample.txt
!cat sample.txt

In [ ]:
%%writefile matrix_input.txt
A,0,0,1
A,0,1,2
A,0,2,3
A,1,0,4
A,1,1,5
A,1,2,6
B,0,0,7
B,0,1,8
B,1,0,9
B,1,1,10
B,2,0,11
B,2,1,12

In [ ]:
%%writefile mapper.py
#!/usr/bin/env python3
import sys
n = 3
p = 2
for line in sys.stdin:
  line = line.strip()
  if not line:
    continue
  name,i,j,val = line.split(',')
  i,j,val = int(i),int(j),float(val)
  if name == 'A':
    for col in range(p):
      print(f"{i},{col}\tA,{j},{val}")
  else:
    for row in range(2):
      print(f"{row},{j}\tB,{i},{val}")

In [ ]:
%%writefile reducer.py
#!/usr/bin/env python3
import sys
from collections import defaultdict
current_key = None
A = defaultdict(float)
B = defaultdict(float)
def emit (key,A,B):
  total = 0
  for k in A:
    total += A[K] * B.get(k,0)
  print(f"{key}\t{total}")
for line in sys.stdin:
  line = line.strip()
  if not line:
    continue
  key,val = line.split('\t')
  if current_key and key != current_key:
    emit(current_key,A,B)
    A = defaultdict(float)
    B = defaultdict(float)
  current_key = key
  tag,k,v= val.split(',')
  k,v = int(k),float(v)
  if tag =='A':
    A[k] = v
  else:
    B[k] = v
if current_key:
  emit(current_key,A,B)

In [ ]:
!chmod +x mapper.py reducer.py

In [ ]:
!hdfs dfs -mkdir /matrix

In [ ]:
!touch matrix.txt


In [ ]:
!hdfs dfs -put matrix.txt /matrix

In [ ]:
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -input matrix_input.txt \
    -output output_matrix \
    -mapper mapper.py \
    -reducer reducer.py
